In [11]:
import numpy as np
import math

data = [['Температура', 'Осадки', 'Месяц', 'Игра'],
        ['> 15', 'Да', 'Апрель', 'Нет'],
        ['< 10', 'Да', 'Март', 'Нет'],
        ['10 − 15', 'Нет', 'Май', 'Да'],
        ['> 15', 'Да', 'Май', 'Нет'],
        ['< 10', 'Нет', 'Апрель', 'Нет'],
        ['> 15', 'Да', 'Март', 'Нет'],
        ['10 − 15', 'Нет', 'Март', 'Да'],
        ['< 10', 'Нет', 'Май', 'Да'],
        ['10 − 15', 'Нет', 'Апрель', 'Нет'],
        ['> 15', 'Нет', 'Май', 'Да'],
        ['< 10', 'Да', 'Март', 'Нет'],
        ['> 15', 'Нет', 'Март', 'Да'],
        ['< 10', 'Нет', 'Март', 'Нет'],
        ['< 10', 'Нет', 'Март', 'Нет']]


def enthropy(data, S, st, HS, cr):
        if st in cr:
                return [-1, -1, -1, -1]
        p = [0]*len(S)
        p0 = [0]*len(S)
        p1 = [0] * len(S)
        for j in range(len(S)):
                for i in range(len(data)):
                        if S[j] == data[i][st]:
                                p[j] += 1
                                if data[i][3] == 'Да':
                                        p0[j] += 1
                                else:
                                        p1[j] += 1
        H = {}
        for i in range(len(S)): #Считаем энтропию
                if p0[i] and p1[i] != 0:
                        h = - p0[i]/p[i] * math.log2(p0[i]/p[i]) - p1[i]/p[i] * math.log2(p1[i]/p[i])
                elif p0[i] == 0:
                        h = - 0 - p1[i] / p[i] * math.log2(p1[i] / p[i])
                else:
                        h = - 0 - p0[i]/p[i] * math.log2(p0[i]/p[i])
                H[S[i]] = h
        I = 0
        for i in range(len(S)): #Информационная выгода
            I += p[i]/len(data) * H[S[i]]
        IG = HS - I
        return [H, IG, p0, p1]

p0 = 0
p1 = 0
p = len(data) - 1
for i in range(1, p):
        if data[i][3] == 'Нет':
                p1 += 1
        else:
                p0 += 1
p0 = p0 / p
p1 = p1 / p
HS = - p0 * math.log2(p0) - p1 * math.log2(p1)  # Энтропия всего набора данных
st = 0
S = {0: []}
while (True): #Заполнение возможных вариантов
        z = []
        for i in range(1, len(data)):
                if data[i][st] not in z:
                        z.append(data[i][st])
        S[st].append(z)
        st += 1
        if st == 3:
                break
        S[st] = []
ID3 = {}
cr = []
key = {}
while(len(data) != 1): #Пока таблица не опустела
        c1 = enthropy(data, S[0][0], 0, HS, cr)
        c2 = enthropy(data, S[1][0], 1, HS, cr)
        c3 = enthropy(data, S[2][0], 2, HS, cr)
        lis = [c1[0], c2[0], c3[0]] #Возможные варианты и их энтропия
        l = [c1[1], c2[1], c3[1]] #Информационная выгода
        p_0 = [c1[2], c2[2], c3[2]]
        p_1 = [c1[3], c2[3], c3[3]]
        m = int(np.argmax(l))
        cr.append(m)
        p_0 = p_0[m]
        p_1 = p_1[m]
        max_I = lis[m] #Наибольшая информационная выгода
        name = data[0][m] #Название
        c = []
        r = []
        for i in (max_I):
                if max_I[i] == 0.0:
                        c.append(str(i)) #Значение
                else:
                        c.append(-100)
                        r.append(i)
        ID3_p = {}
        ID3_p[data[0][m]] = {}
        for i in range(len(c)):
                if p_0[i] == 0:
                        ID3_p[data[0][m]].update({c[i]: 'Нет'})
                if p_1[i] == 0:
                        ID3_p[data[0][m]].update({c[i]: 'Да'})
        for i in range(len(r)):
                ID3_p[data[0][m]].update({r[i]: {}})
                key[m] = [data[0][m]], [r[i]]
        last = data[0][m]
        # Обновление таблицы
        lol = []
        for i in range(len(data)): #Индекс строк, которые нужно удалить
                if data[i][m] in c:
                        lol.append(i)
        for i in range(len(lol)): #Удаление строк
            del data[lol[i]]
            if i != len(lol)-1:
                    lol[i+1] -= i+1

        ID3.update(ID3_p)

ID3[key[2][0][0]][key[2][1][0]].update({last: ID3[last]})
ID3[key[1][0][0]][key[1][1][0]].update({key[2][0][0]: ID3[key[2][0][0]]})
ID3.pop(last)
ID3.pop(key[2][0][0])
print('Дерево', '\n', ID3)
while(True):
        s = list(ID3.keys())
        k1 = list(ID3.keys())[0]
        print(k1, '?')
        print('Выберите вариант:')
        for i in ID3[k1]:
                print(i)
        k2 = str(input())
        if (ID3[k1][k2] == 'Да') or (ID3[k1][k2] == 'Нет'):
                print('Будет ли игра?', ID3[k1][k2])
                break
        ID3 = ID3[k1][k2]


Дерево 
 {'Осадки': {'Да': 'Нет', 'Нет': {'Месяц': {'Апрель': 'Нет', 'Май': 'Да', 'Март': {'Температура': {'> 15': 'Да', '< 10': 'Нет', '10 − 15': 'Да'}}}}}}
Осадки ?
Выберите вариант:
Да
Нет
Нет
Месяц ?
Выберите вариант:
Апрель
Май
Март
Март
Температура ?
Выберите вариант:
> 15
< 10
10 − 15
< 10
Будет ли игра? Нет
